In [89]:
import torch
import torch.nn as nn
import numpy as np
from torch.optim import Adam
from data_rnn import load_ndfa, load_brackets
from utils import device
from random import choices
from tqdm import tqdm

In [90]:
x_train, (i2w, w2i) = load_ndfa(n=150_000)

In [91]:
class DataLoader():
    def __init__(self, x, maxsize=10000, bs=32):
        self.x = [[w2i['.start']] + i + [w2i['.end']] for i in x]
        self.c = 0
        self.maxsize = max(maxsize, len(max(self.x, key=lambda x: len(x))) + 1)
        self.bs = bs
    
    def shuffle(self):
        self.c = 0
        self.x = [self.x[i] for i in np.random.permutation(range(len(self.x)))]
    
    def get(self):
        temp = []
        while len(temp) < self.maxsize:
            if self.c >= len(self.x) or len(temp) + len(self.x[self.c]) + 1 > self.maxsize:
                temp.extend([w2i['.pad']] * (self.maxsize - len(temp)))
            else:
                temp.extend(self.x[self.c] + [w2i['.pad']])
                self.c += 1
        return torch.tensor(temp, dtype=torch.long)
    
    def dataloader(self):
        while self.c < len(self.x):
            x = torch.concat([self.get().view(1, -1) for _ in range(self.bs)]).to(device)
            x = x[x.count_nonzero(dim=1) > 1]
            y = torch.zeros_like(x)
            y[:, :-1] = x[:, 1:]
            yield x, y.view(-1)

In [92]:
class Network(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.emb = nn.Embedding(len(i2w), 32)
        self.lstm = nn.LSTM(32, 16, batch_first=True)
        self.linear = nn.Linear(16, len(w2i))
    
    def forward(self, x):
        x = self.emb(x)
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x.view(-1, len(w2i))

In [97]:
model = Network().to(device)
optimizer = Adam([p for p in model.parameters() if p.requires_grad], lr=3e-4, weight_decay=1e-4)
dl = DataLoader(x_train, bs=8, maxsize=200)
criterion = nn.CrossEntropyLoss()

In [98]:
for epoch in range(10):
    model.train()
    dl.shuffle()
    total_loss = 0
    c = 0
    for x, y in dl.dataloader():
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        loss = criterion(model(x), y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        c += 1
    print(f'Epoch {epoch}, Train Loss: {total_loss / c:.2f}')

Epoch 0, Train Loss: 0.94
Epoch 1, Train Loss: 0.24
Epoch 2, Train Loss: 0.20
Epoch 3, Train Loss: 0.19
Epoch 4, Train Loss: 0.19
Epoch 5, Train Loss: 0.19
Epoch 6, Train Loss: 0.19
Epoch 7, Train Loss: 0.19
Epoch 8, Train Loss: 0.19
Epoch 9, Train Loss: 0.19


In [214]:
sm = nn.Softmax(dim=1)
seq = [w2i['.start'], w2i['s']]
while w2i['.end'] not in seq:
    seq.append(choices(range(len(w2i)), weights=sm(model(torch.tensor([seq], dtype=torch.long, device=device)))[-1, :].detach(), k=1)[0])
print(''.join([i2w[i] for i in seq]))

.startss.end


In [1]:
x_train, (i2w, w2i) = load_brackets(n=150_000)

NameError: name 'load_brackets' is not defined